In [11]:
from dotenv import load_dotenv
import urllib, os, json, pandas as pd, pytz, datetime


def fetch_users():
    endpoint = 'users'
    limit = 200  # max 200
    next_cursor = ''
    df_users = pd.DataFrame()
    while True:
        trocco_api = f'https://trocco.io/api/{endpoint}?limit={limit}&cursor={next_cursor}'
        req = urllib.request.Request(trocco_api)
        req.add_header('Authorization', f'Token {os.getenv("trocco_api_key")}')
        res = json.load(urllib.request.urlopen(req))
        df_users = pd.concat([df_users, pd.DataFrame(res['items'])])
        next_cursor = res.get('next_cursor', '')
        if not next_cursor:
            break
    return df_users

def base_file_name(action):
    jst = pytz.timezone('Asia/Tokyo')
    return f'{datetime.datetime.now(jst).strftime("%Y%m%d")}__{action}_users'

def generate_tfmigrate_hcl_file(action, df_users):
    migration_hcl = f'# tfmigrateでユーザーをまとめて{action}するための定義；ファイル名末尾の.tmpを削除してtfmigrateディレクトリに配置することで処理可能になる\nmigration "state" "{action}_users"' + ' {\n  actions = [\n'
    migration_hcl += '\n'.join([f'    "{action if action != "remove" else "rm"} trocco_user.users[\\\\\\"{email}\\\\\\"]{" " + str(id) if action == "import" else ""}",' for email, id in zip(df_users.email, df_users.id)])
    migration_hcl += '\n  ]\n}\n'
    with open(f'../generated_file/{base_file_name(action)}.hcl.tmp', 'w', encoding='utf-8') as f:
        f.write(migration_hcl)

def generate_users_list(action, df_users):
    if action ==  'import':
        users_list = generate_user_list('imported_users', df_users)
        users_list = users_list + '\n' + generate_user_list('imported_super_admin_user', df_users.query('role == "super_admin"'))
        users_list = users_list + '\n' + generate_user_list('imported_admin_users', df_users.query('role == "admin"'))
        users_list = users_list + '\n' + generate_user_list('imported_connection_modify_not_restricted_users', df_users.query('is_restricted_connection_modify == False'))
        if 'can_use_audit_log' in list(df_users.keys()):
            users_list = users_list + '\n' + generate_user_list('can_use_audit_log_users', df_users.query('can_use_audit_log == True'))
        with open(f'../generated_file/{base_file_name(action)}_list_for_tf.txt', 'w', encoding='utf-8') as f:
            f.write('  # ユーザー取込時に定義に差分が出ないようにするため、権限ごとのユーザー一覧を生成；locals部分に記載する\n' + users_list + '\n')

def generate_user_list(list_name, df_users):
    user_list = f'  {list_name} = [\n'
    user_list += '\n'.join([f'    "{email}",  # {id}' for email, id in zip(df_users.email, df_users.id)])
    user_list += '\n  ]'
    return user_list

def generate_blocks(action, df_users):
    blocks = f'# {action if action != "remove" else "removed"}ブロックでユーザーをまとめて{action}するための定義；terraformのルートディレクトリに配置し、処理後に別フォルダに移管する\n'
    if action == 'import':
        blocks += '\n'.join([f'import {{\n  to = trocco_user.users[\"{email}\"]\n  id = {id}\n}}\n' for email, id in zip(df_users.email, df_users.id)])
    if action == 'remove':
        blocks += '\n'.join([f'removed {{\n  from = trocco_user.users[\"{email}\"]\n  lifecycle {{\n    destroy = false\n  }}\n}}\n' for email, id in zip(df_users.email, df_users.id)])
    with open(f'../generated_file/{base_file_name(action)}.tf.tmp', 'w', encoding='utf-8') as f:
        f.write(blocks)

def main():
    # ルートディレクトリに.envファイルを配置しておく
    load_dotenv()
    df_users = fetch_users()

    if not os.path.isdir('../generated_file'):
        os.makedirs('../generated_file')

    for action in ['import', 'remove']:
        generate_tfmigrate_hcl_file(action, df_users)
        generate_users_list(action, df_users)
        generate_blocks(action, df_users)

if __name__ == '__main__':
    main()